In [1]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4_8ifgmf
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4_8ifgmf
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=8dcbc188f8c76ae931a53200a80651c19eb4b9b4ad39cc3258460542ebec2120
  Stored in directory: /tmp/pip-ephem-wheel-cache-eqis66f_/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
%%cu

#include <iostream>
#include <vector>
#include <ctime>
#include <cstdlib>
#include <cuda_runtime.h>

// Функция для перемножения матриц на CPU
void MatrixMultiplyCPU(const std::vector<float>& A, const std::vector<float>& B, std::vector<float>& C, int N) {
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            float sum = 0.0;
            for (int k = 0; k < N; ++k) {
                sum += A[i * N + k] * B[k * N + j];
            }
            C[i * N + j] = sum;
        }
    }
}

// Функция для перемножения матриц на GPU с использованием CUDA
__global__ void MatrixMultiplyGPU(const float* A, const float* B, float* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sum = 0.0;
    for (int k = 0; k < N; ++k) {
        sum += A[row * N + k] * B[k * N + col];
    }
    C[row * N + col] = sum;
}

int main() {
    int N = 1000; // Размерность матриц (измените на нужное значение)
    std::vector<float> A(N * N);
    std::vector<float> B(N * N);
    std::vector<float> C(N * N);

    // Заполните матрицы A и B случайными данными

    // Выполнение перемножения на CPU
    clock_t cpu_start = clock();
    MatrixMultiplyCPU(A, B, C, N);
    clock_t cpu_end = clock();
    double cpu_time = static_cast<double>(cpu_end - cpu_start) / CLOCKS_PER_SEC;
    std::cout << "CPU время: " << cpu_time << " секунд" << std::endl;

    // Выполнение перемножения на GPU с использованием CUDA
    float* d_A, * d_B, * d_C;
    cudaMalloc((void**)&d_A, sizeof(float) * N * N);
    cudaMalloc((void**)&d_B, sizeof(float) * N * N);
    cudaMalloc((void**)&d_C, sizeof(float) * N * N);

    cudaMemcpy(d_A, A.data(), sizeof(float) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B.data(), sizeof(float) * N * N, cudaMemcpyHostToDevice);

    dim3 block_size(16, 16); // Размер блока на GPU
    dim3 grid_size(N / block_size.x, N / block_size.y);

    cudaEvent_t gpu_start, gpu_end;
    cudaEventCreate(&gpu_start);
    cudaEventCreate(&gpu_end);
    cudaEventRecord(gpu_start);

    MatrixMultiplyGPU<<<grid_size, block_size>>>(d_A, d_B, d_C, N);

    cudaEventRecord(gpu_end);
    cudaEventSynchronize(gpu_end);
    float gpu_time = 0.0;
    cudaEventElapsedTime(&gpu_time, gpu_start, gpu_end);
    gpu_time /= 1000.0; // Преобразование в секунды
    std::cout << "GPU время: " << gpu_time << " секунд" << std::endl;

    cudaMemcpy(C.data(), d_C, sizeof(float) * N * N, cudaMemcpyDeviceToHost);

    // Проверьте корректность результатов (например, сравнив с результатами на CPU)

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


CPU время: 9.17978 секунд
GPU время: 0.00684442 секунд

